In [ ]:
! pip install dash plotly numpy matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
from tqdm import tqdm
import nibabel as nib

In [ ]:
data_path = "data/25226366/cropped/"
patient_no = 3
data_end = np.array(nib.load(data_path + "pat" + str(patient_no) + "_cropped_seg_endpoints.nii.gz").get_fdata())
data_seg = np.array(nib.load(data_path + "pat" + str(patient_no) + "_cropped_seg.nii.gz").get_fdata())
data_cropped = np.array(nib.load(data_path + "pat" + str(patient_no) + "_cropped.nii.gz").get_fdata())
print("endpoint data shape: " + str(data_end.shape))
print("cropped data shape: " + str(data_cropped.shape))
print("segmented data shape: " + str(data_seg.shape))

In [ ]:
print(np.min(data_cropped))

In [ ]:
plt.imshow(data_cropped[40] / np.max(data_cropped), cmap="inferno")

In [ ]:
color_set = set(data_cropped.flatten())

In [ ]:
def tensor_to_3d_points(heart_ten, cropped):
    w, h, l = heart_ten.shape
    color_lim = np.max(cropped)
    scatter_coords = {}
    scatter_colors = {}
    for i in range(w):
        for j in range(h):
            for k in range(l):
                c = heart_ten[i][j][k]
                if c != 0:
                    if c not in scatter_coords:
                        scatter_coords[c] = []
                        scatter_colors[c] = []
                    scatter_coords[c].append([i, j, k])
                    scatter_colors[c].append(cropped[i][j][k] / color_lim)
    return scatter_coords, scatter_colors

In [ ]:
def tensor_to_3d_points_interpolate(heart_ten, cropped):
    w, h, l = heart_ten.shape
    color_lim = np.max(cropped)
    scatter_coords = {}
    scatter_colors = {}
    for i in range(w):
        for j in range(h):
            for k in range(l):
                c = heart_ten[i][j][k]
                if c != 0:
                    if c not in scatter_coords:
                        scatter_coords[c] = []
                        scatter_colors[c] = []
                    scatter_coords[c].append([i, j, k])
                    scatter_colors[c].append(cropped[i][j][k] / color_lim)
    return scatter_coords, scatter_colors

In [ ]:
processed_data, processed_colors = tensor_to_3d_points(data_seg, data_cropped)

In [ ]:
color_keys = list(processed_data.keys())
print(color_keys)
reg_1 = np.array(processed_data[color_keys[0]])
reg_1_colors = np.array(processed_colors[color_keys[0]])
reg_2 = np.array(processed_data[color_keys[1]])
reg_2_colors = np.array(processed_colors[color_keys[1]])
reg_3 = np.array(processed_data[color_keys[2]])
reg_3_colors = np.array(processed_colors[color_keys[2]])
reg_4 = np.array(processed_data[color_keys[3]])
reg_4_colors = np.array(processed_colors[color_keys[3]])
reg_5 = np.array(processed_data[color_keys[4]])
reg5_colors = np.array(processed_colors[color_keys[4]])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
p_size = 1
ax.scatter3D(reg_1[:, 0], reg_1[:, 1], reg_1[:, 2], color="blue", s=p_size)
ax.scatter3D(reg_2[:, 0], reg_2[:, 1], reg_2[:, 2], color="red", s=p_size)
ax.scatter3D(reg_3[:, 0], reg_3[:, 1], reg_3[:, 2], color="orange", s=p_size)
ax.scatter3D(reg_4[:, 0], reg_4[:, 1], reg_4[:, 2], color="orange", s=p_size)
ax.scatter3D(reg_5[:, 0], reg_5[:, 1], reg_5[:, 2], color="purple", s=p_size)
ax.scatter3D(reg_6[:, 0], reg_6[:, 1], reg_6[:, 2], color="green", s=p_size)
ax.view_init(azim=94, elev=15)

In [ ]:
def plot_heart_color_2(voxel_map, colors, az=94, el=15, cmap_name="inferno"):
    fig = go.Figure()

    # Get a Matplotlib colormap to map intensity to colors
    cmap = plt.get_cmap(cmap_name)

    for k in list(voxel_map.keys()):
        reg = np.array(voxel_map[k])   # Voxel positions
        c_arr = np.array(colors[k])    # Intensity values (grayscale)

        # Normalize intensity values to [0, 1]
        c_arr_norm = (c_arr - c_arr.min()) / (c_arr.max() - c_arr.min() + 1e-8)
        
        # Convert grayscale intensity to RGB using colormap
        rgb_colors = cmap(c_arr_norm)[:, :3]  # Extract only RGB channels
        hex_colors = ['rgb({}, {}, {})'.format(int(r * 255), int(g * 255), int(b * 255)) 
                      for r, g, b in rgb_colors]

        # Add scatter plot for each voxel region
        fig.add_trace(go.Scatter3d(
            x=reg[:, 0], y=reg[:, 1], z=reg[:, 2],
            mode='markers',
            marker=dict(size=1.1, color=hex_colors, opacity=1.0)
        ))

    # Set the camera view
    fig.update_layout(
        scene=dict(camera=dict(eye=dict(x=az / 100, y=el / 100, z=2))),
        title="3D Heart Voxel Visualization",
        margin=dict(l=0, r=0, t=40, b=0)
    )
    fig.write_html("raw_vis.html")

    fig.show()


In [ ]:
def plot_heart_color_2(voxel_map, colors, labels, az=94, el=15, cmap_name="inferno"):
    fig = go.Figure()

    # Get a Matplotlib colormap to map intensity to colors
    cmap = plt.get_cmap(cmap_name)

    for k in list(voxel_map.keys()):
        reg = np.array(voxel_map[k])[::2]  # Voxel positions
        c_arr = np.array(colors[k])[::2]    # Intensity values (grayscale)

        # Normalize intensity values to [0, 1]
        c_arr_norm = (c_arr - c_arr.min()) / (c_arr.max() - c_arr.min() + 1e-8)
        
        # Convert grayscale intensity to RGB using colormap
        rgb_colors = cmap(c_arr_norm)[:, :3]  # Extract only RGB channels
        hex_colors = ['rgb({}, {}, {})'.format(int(r * 255), int(g * 255), int(b * 255)) 
                      for r, g, b in rgb_colors]

        # Add scatter plot for each voxel region with labels
        fig.add_trace(go.Scatter3d(
            x=reg[:, 0], y=reg[:, 1], z=reg[:, 2],
            mode='markers',
            marker=dict(size=2, color=hex_colors, opacity=1.0),
            name=labels[k]  # Set the legend label
        ))

    # Set the camera view
    fig.update_layout(
        scene=dict(camera=dict(eye=dict(x=az / 100, y=el / 100, z=2))),
        title="3D Heart Voxel Visualization",
        margin=dict(l=0, r=0, t=40, b=0),
        legend=dict(
        x=1, y=1,
        font=dict(size=14),  # Increase font size
        itemsizing='constant'  # Makes the legend items larger
    )
    )
    fig.write_html("raw_vis.html")

    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

def is_surface_voxel(voxel_positions, vox_colors):
    """ Identify surface voxels by checking if they have any missing neighbors. """
    voxel_set = {tuple(voxel) for voxel in voxel_positions}  # Convert to a set for fast lookup
    surface_voxels = []
    surface_colors = []
    # 6-connectivity (checking direct neighbors in a grid)
    neighbors = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
    
    i = 0
    for voxel in voxel_positions:
        if any(tuple(np.array(voxel) + np.array(n)) not in voxel_set for n in neighbors):
            surface_voxels.append(voxel)
            surface_colors.append(vox_colors[i])
        else:
            print("pruning")
        i += 1
    return np.array(surface_voxels), np.array(surface_colors)

def plot_heart_color_2(voxel_map, colors, labels, az=94, el=15, cmap_name="inferno"):
    fig = go.Figure()

    # Get a Matplotlib colormap to map intensity to colors
    cmap = plt.get_cmap(cmap_name)

    for k in list(voxel_map.keys()):
        reg = np.array(voxel_map[k])  # Original voxel positions
        c_arr = np.array(colors[k])  # Intensity values (grayscale)

        # Keep only surface voxels
        surface_reg, surface_c_arr = is_surface_voxel(reg, c_arr)

        # Normalize intensity values to [0, 1]
        c_arr_norm = (surface_c_arr - surface_c_arr.min()) / (surface_c_arr.max() - surface_c_arr.min() + 1e-8)
        
        # Convert grayscale intensity to RGB using colormap
        rgb_colors = cmap(c_arr_norm)[:, :3]  # Extract only RGB channels
        hex_colors = ['rgb({}, {}, {})'.format(int(r * 255), int(g * 255), int(b * 255)) 
                      for r, g, b in rgb_colors]

        # Add scatter plot for each voxel region with labels
        fig.add_trace(go.Scatter3d(
            x=surface_reg[:, 0], y=surface_reg[:, 1], z=surface_reg[:, 2],
            mode='markers',
            marker=dict(size=1.2, color=hex_colors, opacity=1.0),
            name=labels[k]  # Set the legend label
        ))

    # Set the camera view
    fig.update_layout(
        scene=dict(camera=dict(eye=dict(x=az / 100, y=el / 100, z=2))),
        title="3D Heart Voxel Visualization",
        margin=dict(l=0, r=0, t=40, b=0),
        legend=dict(
            x=1, y=1,
            font=dict(size=14),  # Increase font size
            itemsizing='constant'  # Makes the legend items larger
        )
    )
    fig.write_html("raw_vis.html")

    fig.show()


In [ ]:
cols_map = {
    1.0: "Left Ventricle", 
    2.0: "Right Ventricle",
    3.0: "Left Atrium",
    4.0: "Right Atrium",
    5.0: "Aorta",
    6.0: "Pulmonary Artery",
    7.0: "Superior Vena Cava",
    8.0: "Inferior Vena Cava"
    }
plot_heart_color_2(processed_data, processed_colors, cols_map)

In [ ]:
def voxel_map_to_mesh(voxel_map, colors, folder="meshes", cmap_name="inferno"):
    """
    Converts each segmented volume in voxel_map into a smooth mesh.
    """
    import os
    os.makedirs(folder, exist_ok=True)
    
    cmap = plt.get_cmap(cmap_name)
    
    for key in voxel_map.keys():
        points = np.array(voxel_map[key])
        c_arr = np.array(colors[key])

        # Normalize intensity values
        c_arr_norm = (c_arr - c_arr.min()) / (c_arr.max() - c_arr.min() + 1e-8)
        rgb_colors = (cmap(c_arr_norm)[:, :3] * 255).astype(np.uint8)

        # Convert to Open3D format
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)
        point_cloud.colors = o3d.utility.Vector3dVector(rgb_colors / 255.0)

        # Remove outliers to avoid internal points
        #point_cloud = remove_outliers(point_cloud)

        # Compute better normals
        #point_cloud = compute_normals(point_cloud)

        # Generate smooth mesh using Poisson reconstruction
        mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(point_cloud, depth=8)

        # Smooth the mesh to remove sharp artifacts
        mesh = mesh.filter_smooth_laplacian(10)

        # Save mesh
        mesh_filename = f"{folder}/heart_region_{key}.ply"
        o3d.io.write_triangle_mesh(mesh_filename, mesh)
        print(f"Saved mesh for region {key} as '{mesh_filename}'")

        # Show the mesh
        o3d.visualization.draw_geometries([mesh])

# Example usage:
# voxel_map_to_mesh(voxel_map, colors, "heart_meshes")


In [ ]:
def plot_heart_color(voxel_map, colors, az=94, el=15):
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(projection='3d')
    p_size = 5
    for k in voxel_map.keys():
        reg = np.array(voxel_map[k])
        c_arr = np.array(colors[k])
        ax.scatter3D(reg[:, 0], reg[:, 1], reg[:, 2], c=c_arr, s=p_size, cmap="inferno")
    ax.view_init(azim=az, elev=el)

In [ ]:
def plot_heart_color_3(voxel_map, colors, az=94, el=15, cmap_name="inferno", save_as="plot.html"):
    fig = go.Figure()

    import matplotlib.pyplot as plt
    cmap = plt.get_cmap(cmap_name)

    for k in list(voxel_map.keys()):
        reg = np.array(voxel_map[k])   # Voxel positions
        c_arr = np.array(colors[k])    # Intensity values (grayscale)

        # Normalize intensity values to [0, 1]
        c_arr_norm = (c_arr - c_arr.min()) / (c_arr.max() - c_arr.min() + 1e-8)
        
        # Convert grayscale intensity to RGB using colormap
        rgb_colors = cmap(c_arr_norm)[:, :3]  # Extract only RGB channels
        hex_colors = ['rgb({}, {}, {})'.format(int(r * 255), int(g * 255), int(b * 255)) 
                      for r, g, b in rgb_colors]

        # Add scatter plot for each voxel region
        fig.add_trace(go.Scatter3d(
            x=reg[:, 0], y=reg[:, 1], z=reg[:, 2],
            mode='markers',
            marker=dict(size=4, color=hex_colors, opacity=1.0)
        ))

    # Set the camera view
    fig.update_layout(
        scene=dict(camera=dict(eye=dict(x=az / 100, y=el / 100, z=2))),
        title="3D Heart Voxel Visualization",
        margin=dict(l=0, r=0, t=40, b=0)
    )

    # Save to HTML
    pio.write_html(fig, save_as)
    print(f"Plot saved as '{save_as}'")

In [ ]:
plot_heart_color_3(processed_data, processed_colors)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes

def plot_smooth_heart(voxel_map, colors, az=94, el=15, cmap_name="inferno", export_html="smooth_heart.html"):
    fig = go.Figure()
    cmap = plt.get_cmap(cmap_name)

    for k in voxel_map.keys():
        reg = np.array(voxel_map[k])   # Voxel positions
        c_arr = np.array(colors[k])    # Intensity values

        # Normalize intensity values
        c_arr_norm = (c_arr - c_arr.min()) / (c_arr.max() - c_arr.min() + 1e-8)
        rgb_colors = cmap(c_arr_norm)[:, :3]  # Convert grayscale to RGB
        
        # Create a 3D binary grid for marching cubes
        voxel_grid = np.zeros((reg[:, 0].max()+1, reg[:, 1].max()+1, reg[:, 2].max()+1))
        voxel_grid[reg[:, 0], reg[:, 1], reg[:, 2]] = 1  # Set occupied voxels

        # Compute mesh using marching cubes
        verts, faces, _, _ = marching_cubes(voxel_grid, level=0.1)

        # Convert colors for faces (average of neighboring vertices)
        face_colors = np.mean(rgb_colors, axis=0)

        # Convert face colors to RGB format
        mesh_colors = 'rgb({}, {}, {})'.format(int(face_colors[0] * 255),
                                               int(face_colors[1] * 255),
                                               int(face_colors[2] * 255))

        # Add smooth surface mesh
        fig.add_trace(go.Mesh3d(
            x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            color=mesh_colors,
            opacity=1.0
        ))

    # Set camera view
    fig.update_layout(
        scene=dict(camera=dict(eye=dict(x=az / 100, y=el / 100, z=2))),
        title="Smooth 3D Heart Visualization",
        margin=dict(l=0, r=0, t=40, b=0)
    )
    fig.write_html(export_html)
    print(f"Interactive plot saved as {export_html}")
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes

def plot_smooth_heart(voxel_map, az=94, el=15, cmap_name="inferno", export_html="smooth_heart.html"):
    fig = go.Figure()
    cmap = plt.get_cmap(cmap_name)

    # Assign unique colors per key in voxel_map
    unique_keys = list(voxel_map.keys())
    key_colors = {k: cmap(i / len(unique_keys))[:3] for i, k in enumerate(unique_keys)}

    for k in unique_keys:
        reg = np.array(voxel_map[k])  # Voxel positions

        # Create a 3D binary grid for marching cubes
        voxel_grid = np.zeros((reg[:, 0].max() + 1, reg[:, 1].max() + 1, reg[:, 2].max() + 1))
        voxel_grid[reg[:, 0], reg[:, 1], reg[:, 2]] = 1  # Set occupied voxels

        # Compute mesh using marching cubes
        verts, faces, _, _ = marching_cubes(voxel_grid, level=0.5)

        # Assign the same color to the entire region
        region_color = key_colors[k]
        mesh_colors = 'rgb({}, {}, {})'.format(
            int(region_color[0] * 255),
            int(region_color[1] * 255),
            int(region_color[2] * 255)
        )

        # Add smooth surface mesh
        fig.add_trace(go.Mesh3d(
            x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            color=mesh_colors,
            opacity=1.0,
            name=f"Region {k}"
        ))

    # Set camera view
    fig.update_layout(
        title="Smooth 3D Heart Visualization",
        margin=dict(l=0, r=0, t=40, b=0)
    )

    # Save interactive HTML
    fig.write_html(export_html)
    print(f"Interactive plot saved as {export_html}")
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes

def plot_smooth_heart(voxel_map, az=94, el=15, cmap_name="turbo", export_html="smooth_heart.html"):
    fig = go.Figure()
    cmap = plt.get_cmap(cmap_name)

    # Get unique keys and determine intensity scaling
    unique_keys = list(voxel_map.keys())
    num_regions = len(unique_keys)
    
    # Assign colors based on normalized index in colormap
    key_colors = {k: cmap(i / (num_regions - 1 if num_regions > 1 else 1))[:3] for i, k in enumerate(unique_keys)}

    for k in unique_keys:
        reg = np.array(voxel_map[k])  # Voxel positions
        
        if reg.shape[0] == 0:
            continue  # Skip empty regions
        
        # Define voxel grid size dynamically based on max indices
        grid_size = np.max(reg, axis=0) + 2  # Add padding to avoid boundary issues
        voxel_grid = np.zeros(grid_size, dtype=np.uint8)
        voxel_grid[reg[:, 0], reg[:, 1], reg[:, 2]] = 1  # Set occupied voxels

        # Compute mesh using marching cubes
        verts, faces, _, _ = marching_cubes(voxel_grid, level=0.5)

        # Assign a unique smooth color per region
        region_color = key_colors[k]
        mesh_colors = 'rgb({}, {}, {})'.format(
            int(region_color[0] * 255),
            int(region_color[1] * 255),
            int(region_color[2] * 255)
        )

        # Add smooth surface mesh with toggleable traces
        fig.add_trace(go.Mesh3d(
            x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            color=mesh_colors,
            opacity=1.0,  # Slight transparency for better depth perception
            lighting=dict(ambient=0.8, diffuse=0.8),
            name=f"Region {k}",  # Legend entry
            visible=True  # Allows toggling
        ))

    # Set camera view and add interactive legend
    fig.update_layout(
        title="Smooth 3D Heart Visualization",
        showlegend=True  # Enables trace selection in legend
    )

    # Save interactive HTML
    fig.write_html(export_html)
    print(f"Interactive plot saved as {export_html}")
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes
from scipy.ndimage import gaussian_filter

def plot_smooth_heart(voxel_map, az=94, el=15, cmap_name="RdYlBu", sigma=1.0, export_html="smooth_heart.html"):
    fig = go.Figure()
    cmap = plt.get_cmap(cmap_name)

    # Get unique keys and determine intensity scaling
    unique_keys = list(voxel_map.keys())
    num_regions = len(unique_keys)
    
    # Assign colors based on normalized index in colormap
    key_colors = {k: cmap(i / (num_regions - 1 if num_regions > 1 else 1))[:3] for i, k in enumerate(unique_keys)}

    for k in unique_keys:
        reg = np.array(voxel_map[k])  # Voxel positions
        
        if reg.shape[0] == 0:
            continue  # Skip empty regions
        
        # Define voxel grid size dynamically based on max indices
        grid_size = np.max(reg, axis=0) + 3  # Add padding to avoid boundary issues
        voxel_grid = np.zeros(grid_size, dtype=np.float32)
        voxel_grid[reg[:, 0], reg[:, 1], reg[:, 2]] = 1  # Set occupied voxels

        # 🔹 Apply Gaussian Smoothing for softer edges
        voxel_grid = gaussian_filter(voxel_grid, sigma=sigma)

        # Compute mesh using marching cubes
        verts, faces, _, _ = marching_cubes(voxel_grid, level=0.3)  # Lower threshold for better smoothing

        # Assign a unique smooth color per region
        region_color = key_colors[k]
        mesh_colors = 'rgb({}, {}, {})'.format(
            int(region_color[0] * 255),
            int(region_color[1] * 255),
            int(region_color[2] * 255)
        )

        # Add smooth surface mesh with toggleable traces
        fig.add_trace(go.Mesh3d(
            x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            color=mesh_colors,
            opacity=1.0,  # Slight transparency for depth perception
            lighting=dict(ambient=0.8, diffuse=0.8),
            name=f"Region {k}",  # Legend entry
            visible=True  # Allows toggling
        ))

    # Set camera view and add interactive legend
    fig.update_layout(
        title="Smooth 3D Heart Visualization",
        showlegend=True  # Enables trace selection in legend
    )

    # Save interactive HTML
    fig.write_html(export_html)
    print(f"Interactive plot saved as {export_html}")
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes
from scipy.ndimage import gaussian_filter

def plot_smooth_heart(voxel_map, labels, az=94, el=15, cmap_name="RdYlBu", sigma=1.0, export_html="smooth_heart.html"):
    fig = go.Figure()
    cmap = plt.get_cmap(cmap_name)

    # Get unique keys and determine intensity scaling
    unique_keys = list(voxel_map.keys())
    num_regions = len(unique_keys)
    
    # Assign colors based on normalized index in colormap
    key_colors = {k: cmap(i / (num_regions - 1 if num_regions > 1 else 1))[:3] for i, k in enumerate(unique_keys)}

    for k in unique_keys:
        reg = np.array(voxel_map[k])  # Voxel positions
        
        if reg.shape[0] == 0:
            continue  # Skip empty regions
        
        # Define voxel grid size dynamically based on max indices
        grid_size = np.max(reg, axis=0) + 3  # Add padding to avoid boundary issues
        voxel_grid = np.zeros(grid_size, dtype=np.float32)
        voxel_grid[reg[:, 0], reg[:, 1], reg[:, 2]] = 1  # Set occupied voxels

        # 🔹 Apply Gaussian Smoothing for softer edges
        voxel_grid = gaussian_filter(voxel_grid, sigma=sigma)

        # Compute mesh using marching cubes
        verts, faces, _, _ = marching_cubes(voxel_grid, level=0.3)  # Lower threshold for better smoothing

        # Assign a unique smooth color per region
        region_color = key_colors[k]
        mesh_colors = 'rgb({}, {}, {})'.format(
            int(region_color[0] * 255),
            int(region_color[1] * 255),
            int(region_color[2] * 255)
        )

        # Add smooth surface mesh (but it won't appear in the legend)
        fig.add_trace(go.Mesh3d(
            x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            color=mesh_colors,
            opacity=1.0,
            lighting=dict(ambient=0.8, diffuse=0.8),
            hoverinfo="skip"  # Avoids cluttering hover labels
        ))

        # **Invisible Scatter3D Marker for the Legend**
        fig.add_trace(go.Scatter3d(
            x=[None], y=[None], z=[None],  # Invisible point
            mode="markers",
            marker=dict(size=10, color=mesh_colors),
            name=labels.get(k, f"Region {k}")  # Legend entry
        ))

    # Set camera view and add interactive legend
    fig.update_layout(
        title="Smooth 3D Heart Visualization",
        showlegend=True,
        legend=dict(x=1, y=1),  # Place legend in top-right
    )

    # Save interactive HTML
    fig.write_html(export_html)
    print(f"Interactive plot saved as {export_html}")
    fig.show()


In [ ]:
plot_smooth_heart(processed_data, cols_map, export_html="pat3_unhealthy_52yo_segmented.html")